In [1]:
import os
import sys
import yaml
import json

with open("config.yaml", 'r') as yml_file:
    cfg = yaml.safe_load(yml_file)
    
#Get data path from config 
data_path = cfg['brian_laptop']['data_path']
mb_div = cfg['constant']['mb_div']

#Get dyype for meta file 
meta_data_file = os.path.join(data_path, cfg['data']['metadata'])
with open(meta_data_file) as f:
    meta_data = json.load(f)

In [2]:
import datatable as dt
print(dt.__version__)

0.9.0


# Open Files
- no support for hdf, parquet files yet 

In [3]:
#CSV use fread command
yellow_trip_file = os.path.join(data_path, cfg['data']['nyc_yellow_trip_csv'])
yellow_taxi = dt.fread(yellow_trip_file, fill = True)

In [4]:
# Had introduced extra line on top row 
# Columns are colours by type 
yellow_taxi.head(2)

,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,…,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,NA,NA,NA,NA,NA,NA,NA,NA,NA,NA,…,NA,NA,NA,NA,NA
1,CMT,2010-06-18 10:43:46,2010-06-18 11:22:12,4,20,−73.9729,40.7928,2,0,−73.7763,…,0,0.5,0,4.57,50.07


In [5]:
#Fread has introduced extra row
print(yellow_taxi.nrows)
x = yellow_taxi.nrows
yellow_taxi = yellow_taxi[1:x,:]
print(yellow_taxi.nrows)
yellow_taxi.head(2)

14825129
14825128


,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,dropoff_longitude,…,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪,▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,CMT,2010-06-18 10:43:46,2010-06-18 11:22:12,4,20,−73.9729,40.7928,2,0,−73.7763,…,0,0.5,0,4.57,50.07
1,CMT,2010-06-26 15:02:57,2010-06-26 15:07:15,1,0.7,−73.9878,40.7485,1,0,−73.9775,…,0,0.5,0,0,5


In [6]:
for i in range(len(yellow_taxi.names)):
    print(yellow_taxi.names[i], ":", yellow_taxi.stypes[i])

vendor_id : stype.str32
pickup_datetime : stype.str32
dropoff_datetime : stype.str32
passenger_count : stype.int32
trip_distance : stype.float64
pickup_longitude : stype.float64
pickup_latitude : stype.float64
rate_code : stype.int32
store_and_fwd_flag : stype.bool8
dropoff_longitude : stype.float64
dropoff_latitude : stype.float64
payment_type : stype.str32
fare_amount : stype.float64
surcharge : stype.float64
mta_tax : stype.float64
tip_amount : stype.float64
tolls_amount : stype.float64
total_amount : stype.float64


## Summary
- nice guide here https://www.machinelearningplus.com/data-manipulation/101-python-datatable-exercises-pydatatable/
- no describe method, but could be written. 
- does nice caching

In [7]:
stat_dt = dt.Frame(stat=["sum","max","min","mean","sd","mode","nmodal","nuniqe"])

In [8]:
result = dt.rbind(yellow_taxi.sum(),
yellow_taxi.max(),
yellow_taxi.min(),
yellow_taxi.mean(),
yellow_taxi.sd(),
yellow_taxi.mode(),
yellow_taxi.nmodal(),
yellow_taxi.nunique())

result = dt.cbind(stat_dt, result)
result

,stat,vendor_id,pickup_datetime,dropoff_datetime,passenger_count,trip_distance,pickup_longitude,pickup_latitude,rate_code,store_and_fwd_flag,…,surcharge,mta_tax,tip_amount,tolls_amount,total_amount
,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,sum,NA,NA,NA,2.47952e+07,4.15814e+07,−1.07333e+09,5.91892e+08,1.5342e+07,84544,…,4.87368e+06,7.38679e+06,1.17109e+07,2.80826e+06,1.76881e+08
1,max,NA,NA,NA,208,50,3569.93,1963.52,210,1,…,8.05,0.5,100,20,235.9
2,min,NA,NA,NA,0,0,−884.851,−3579.14,0,0,…,0,0,0,0,2.5
3,mean,NA,NA,NA,1.67251,2.80479,−72.399,39.9249,1.03486,0.0126752,…,0.328744,0.498261,0.789939,0.189425,11.9311
4,sd,NA,NA,NA,1.28233,3.23032,10.7638,10.6181,0.350489,0.111868,…,0.369467,0.0294337,1.5956,0.963193,9.38099
5,mode,VTS,2010-06-18 00:00:00,2010-06-22 19:44:00,1,1,0,0,1,0,…,0,0.5,0,0,7
6,nmodal,7471719,380,363,1.02076e+07,336853,312492,312449,1.44822e+07,6.58548e+06,…,7.45368e+06,1.47736e+07,9.57667e+06,1.42418e+07,304454
7,nuniqe,3,2260407,2262482,10,3921,216639,215251,12,2,…,26,2,2615,528,10132


## Value Count

### f-expressions

The datatable module exports a special symbol f, which can be used to refer to the columns of a frame currently being operated on. If this sounds cryptic, consider that the most common way to operate on a frame is via the square-bracket call DT[i, j, by, ...]. And it is often the case that within this expression you would want to refer to individual columns of the frame: either to create a filter, or a transform, or specify a grouping variable, etc. In all such cases the f symbol is used, and it is considered to be evaluated within the context of the frame DT.

In [9]:
yellow_taxi[:, dt.count(), dt.by("passenger_count")]

,passenger_count,C0
,▪▪▪▪,▪▪▪▪
0,0,39
1,1,10207628
2,2,2293152
3,3,680684
4,4,323814
5,5,1262078
6,6,57696
7,9,1
8,133,1


In [10]:
#yellow_taxi is the frame, dt is the library, dt.f is the special symbol
yellow_taxi[:, {"mean_tot_amount" :dt.mean(dt.f.total_amount)}, dt.by("passenger_count")]

,passenger_count,mean_tot_amount
,▪▪▪▪,▪▪▪▪▪▪▪▪
0,0,17.4133
1,1,11.8145
2,2,12.3407
3,3,12.0267
4,4,12.1366
5,5,12.0404
6,6,11.6195
7,9,14.7
8,133,10.3


## Dates

In [11]:
#TODO 
#Can't work out how to do this with datetime. 

## Groupby 

In [12]:
result = yellow_taxi[:, {"fare_amount" : dt.mean(dt.f.fare_amount),
                        "tip_amount" : dt.mean(dt.f.tip_amount)},
                     dt.by("vendor_id")]

result

,vendor_id,fare_amount,tip_amount
,▪▪▪▪,▪▪▪▪▪▪▪▪,▪▪▪▪▪▪▪▪
0,CMT,9.93636,0.74083
1,DDS,10.8225,0.771464
2,VTS,10.2244,0.835468


## PyDataTable Interesting Bits

### Machine Learning 

- TODO: FTRL is implemented in pydatatable

## Observations

- pydatatable is trying to mimick that data.table in R. This is confusing 
- documentation is poor 